In [91]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from path import Path
from sklearn import tree

from collections import Counter

In [92]:
# Reading the file into Dataframe
df_loan = pd.read_csv('https://finalprojectanalysis-bucket.s3.us-east-2.amazonaws.com/clean_data.csv')
df_loan.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [93]:
# Identifying the outliers,spread (for scaling)
df_loan.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,505.000000,505.000000,505.000000,505.000000,505.000000
mean,5411.427723,1562.978059,144.566337,341.869307,0.859406
std,6127.874416,2564.980770,81.306177,64.900341,0.347947
min,150.000000,0.000000,9.000000,36.000000,0.000000
25%,2895.000000,0.000000,100.000000,360.000000,1.000000
50%,3833.000000,1126.000000,128.000000,360.000000,1.000000
75%,5818.000000,2253.000000,167.000000,360.000000,1.000000
max,81000.000000,33837.000000,600.000000,480.000000,1.000000


In [94]:
# Removing outliers
df_loan = df_loan.loc[(df_loan["ApplicantIncome"] < 50000)] 

In [95]:
df_loan = df_loan.loc[(df_loan["ApplicantIncome"] > 1000)]

In [96]:
df_loan.shape

(499, 12)

In [97]:
# Creating a new column
df_loan ['Total_Income'] = df_loan ['ApplicantIncome'] + df_loan ['CoapplicantIncome']

In [98]:
#Remving dpendents
#df_loan.drop("Dependents", axis=1, inplace=True)

In [99]:
df_loan.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Total_Income
0,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,6091.0
1,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,3000.0
2,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,4941.0
3,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,6000.0
4,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y,9613.0


In [100]:
#pd.getdummies to encode
df_loan_encoded = pd.get_dummies(df_loan, columns=["Gender","Education","Dependents","Property_Area","Self_Employed","Married"])

In [101]:
# Creating Encode instancce 
lbl_encode = LabelEncoder()

In [102]:
# Encode Loan_Status Column
df_loan_encoded['Loan_Status_label']=lbl_encode.fit_transform(df_loan_encoded['Loan_Status'])

In [103]:
df_loan_encoded.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Total_Income,Gender_Female,Gender_Male,Education_Graduate,...,Dependents_3+,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Self_Employed_No,Self_Employed_Other,Self_Employed_Yes,Married_No,Married_Yes,Loan_Status_label
0,4583,1508.0,128.0,360.0,1.0,N,6091.0,0,1,1,...,0,1,0,0,1,0,0,0,1,0
1,3000,0.0,66.0,360.0,1.0,Y,3000.0,0,1,1,...,0,0,0,1,0,0,1,0,1,1
2,2583,2358.0,120.0,360.0,1.0,Y,4941.0,0,1,0,...,0,0,0,1,1,0,0,0,1,1
3,6000,0.0,141.0,360.0,1.0,Y,6000.0,0,1,1,...,0,0,0,1,1,0,0,1,0,1
4,5417,4196.0,267.0,360.0,1.0,Y,9613.0,0,1,1,...,0,0,0,1,0,0,1,0,1,1


In [104]:
# Define features
y = df_loan_encoded["Loan_Status_label"]
X = df_loan_encoded.drop(columns=['Loan_Status','Loan_Status_label'])

In [105]:
Counter(y)

Counter({0: 149, 1: 350})

In [106]:
# Train test split (stratify=y to avoid worsening of the class imbalance)
X_train, X_test, y_train, y_test = train_test_split(X,
   y, random_state=1, stratify=y)

In [107]:
# Resample using Oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [108]:
# Resample
#from imblearn.combine import SMOTEENN
#smote_enn = SMOTEENN(random_state=1)
#X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)

In [109]:
# Looking at Class imbalance (Mild imbalance 30%)
Counter(y_resampled)

Counter({1: 262, 0: 262})

In [110]:
#Checking Class imbalance in y_train
Counter(y_train)

Counter({1: 262, 0: 112})

In [111]:
Counter (y_test)

Counter({1: 88, 0: 37})

In [112]:
# Stadardizing and scaling
from sklearn.preprocessing import StandardScaler
# Creating stadard scaler instance
sc = StandardScaler()

# Fit and transform
X_train_scaled = sc.fit_transform(X_resampled)
X_test_scaled = sc.transform(X_test)

In [113]:
# Insantiate model  SKLearn
classifier = LogisticRegression(solver='lbfgs',
   max_iter=400,
   random_state=1)


In [114]:
X_train.shape

(374, 22)

In [115]:
# Train the model
classifier.fit(X_train_scaled, y_resampled)

LogisticRegression(max_iter=400, random_state=1)

In [116]:
y_pred = classifier.predict(X_test_scaled)

In [117]:
Counter (y_pred)

Counter({1: 89, 0: 36})

In [118]:
Counter (y_resampled)

Counter({1: 262, 0: 262})

In [119]:
# Print results
print("Accuracy Score")
print(accuracy_score(y_test, y_pred))
print ("-----------------------------")

print("Confusion Matrix")
print ("                             ")
print(pd.DataFrame(
    confusion_matrix(y_test, y_pred), index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
)
print ("-----------------------------")

print("Classification Report")
print ("                             ")
print(classification_report(y_test, y_pred))

Accuracy Score
0.8
-----------------------------
Confusion Matrix
                             
          Predicted 0  Predicted 1
Actual 0           24           13
Actual 1           12           76
-----------------------------
Classification Report
                             
              precision    recall  f1-score   support

           0       0.67      0.65      0.66        37
           1       0.85      0.86      0.86        88

    accuracy                           0.80       125
   macro avg       0.76      0.76      0.76       125
weighted avg       0.80      0.80      0.80       125



In [120]:
## DECISION TREE MODEL

In [121]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_resampled)

In [122]:
# Making predictions using the testing data.
y_pred = model.predict(X_test_scaled)

In [123]:
# Print results
print("Accuracy Score")
print(accuracy_score(y_test, y_pred))
print ("-----------------------------")

print("Confusion Matrix")
print ("                             ")
print(pd.DataFrame(
    confusion_matrix(y_test, y_pred), index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
)
print ("-----------------------------")

print("Classification Report")
print ("                             ")
print(classification_report(y_test, y_pred))

Accuracy Score
0.76
-----------------------------
Confusion Matrix
                             
          Predicted 0  Predicted 1
Actual 0           20           17
Actual 1           13           75
-----------------------------
Classification Report
                             
              precision    recall  f1-score   support

           0       0.61      0.54      0.57        37
           1       0.82      0.85      0.83        88

    accuracy                           0.76       125
   macro avg       0.71      0.70      0.70       125
weighted avg       0.75      0.76      0.76       125



In [124]:
#random forest


In [125]:
# Create a random forest classifier.
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=1) 

In [126]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_resampled)

In [127]:
# Making predictions using the testing data.
y_pred = rf_model.predict(X_test_scaled)

In [128]:
# Calculating the accuracy score.
acc_score=print(accuracy_score(y_test, y_pred))

0.792


In [129]:

# Print results
print("Accuracy Score")
print(accuracy_score(y_test, y_pred))
print ("-----------------------------")

print("Confusion Matrix")
print ("                             ")
print(pd.DataFrame(
    confusion_matrix(y_test, y_pred), index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
)
print ("-----------------------------")

print("Classification Report")
print ("                             ")
print(classification_report(y_test, y_pred))

Accuracy Score
0.792
-----------------------------
Confusion Matrix
                             
          Predicted 0  Predicted 1
Actual 0           23           14
Actual 1           12           76
-----------------------------
Classification Report
                             
              precision    recall  f1-score   support

           0       0.66      0.62      0.64        37
           1       0.84      0.86      0.85        88

    accuracy                           0.79       125
   macro avg       0.75      0.74      0.75       125
weighted avg       0.79      0.79      0.79       125



In [130]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.1666071222758066, 'Credit_History'),
 (0.16012304895830146, 'Total_Income'),
 (0.14444008589919285, 'LoanAmount'),
 (0.14415092019690118, 'ApplicantIncome'),
 (0.0958073472013327, 'CoapplicantIncome'),
 (0.04173730412401029, 'Loan_Amount_Term'),
 (0.03597635379423138, 'Property_Area_Semiurban'),
 (0.02200378736446486, 'Property_Area_Rural'),
 (0.018566920013811893, 'Dependents_0'),
 (0.016804350531307105, 'Education_Graduate'),
 (0.01679746096148335, 'Property_Area_Urban'),
 (0.016427238026239136, 'Married_No'),
 (0.014926652878803402, 'Gender_Female'),
 (0.013965683577088125, 'Self_Employed_No'),
 (0.013856689687567775, 'Married_Yes'),
 (0.013625131399239312, 'Gender_Male'),
 (0.013541688629749462, 'Education_Not Graduate'),
 (0.013230642084325525, 'Dependents_2'),
 (0.012620909447917278, 'Self_Employed_Yes'),
 (0.010966171545723042, 'Dependents_1'),
 (0.008369667437917743, 'Dependents_3+'),
 (0.005454823964585435, 'Self_Employed_Other')]

In [131]:
# Gradient BOOST

In [132]:
Counter (y_resampled)

Counter({1: 262, 0: 262})

In [133]:
X_train_scaled

array([[-0.63510197, -0.13503653, -0.58928891, ..., -0.4055175 ,
        -0.81455022,  0.81455022],
       [ 3.58946441,  1.37855433,  4.9193644 , ..., -0.4055175 ,
        -0.81455022,  0.81455022],
       [-0.64552564,  0.20320431, -0.61258131, ..., -0.4055175 ,
        -0.81455022,  0.81455022],
       ...,
       [-0.56263261,  0.04922349, -0.5543503 , ..., -0.4055175 ,
        -0.81455022,  0.81455022],
       [-0.41446752,  0.23052945, -0.33307247, ..., -0.4055175 ,
        -0.81455022,  0.81455022],
       [-0.27325155, -0.56596128, -0.36801108, ..., -0.4055175 ,
         1.22767139, -1.22767139]])

In [134]:
from sklearn.ensemble import GradientBoostingClassifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=learning_rate,
   max_features=5,
   max_depth=3,
   random_state=1)
    classifier.fit(X_train_scaled, y_resampled)
    
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(
       classifier.score(
           X_train_scaled,
           y_resampled)))
    print("Accuracy score (validation): {0:.3f}".format(
       classifier.score(
           X_test_scaled,
           y_test)))
    

Learning rate:  0.05
Accuracy score (training): 0.784
Accuracy score (validation): 0.832
Learning rate:  0.1
Accuracy score (training): 0.815
Accuracy score (validation): 0.824
Learning rate:  0.25
Accuracy score (training): 0.874
Accuracy score (validation): 0.792
Learning rate:  0.5
Accuracy score (training): 0.931
Accuracy score (validation): 0.776
Learning rate:  0.75
Accuracy score (training): 0.952
Accuracy score (validation): 0.768
Learning rate:  1
Accuracy score (training): 0.956
Accuracy score (validation): 0.720


In [135]:
classifier = GradientBoostingClassifier(n_estimators=40,
   learning_rate=0.05, max_features=5, max_depth=3, random_state=1)

classifier.fit(X_train_scaled, y_resampled)
y_pred = classifier.predict(X_test_scaled)

In [136]:
# Calculating the confusion matrix
#cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
#cm_df = pd.DataFrame(
   # confusion_matrix(y_test, y_pred), index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

#cm_df

In [137]:
# Print results
print("Accuracy Score")
print(accuracy_score(y_test, y_pred))
print ("-----------------------------")

print("Confusion Matrix")
print ("                             ")
print(pd.DataFrame(
    confusion_matrix(y_test, y_pred), index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
)
print ("-----------------------------")

print("Classification Report")
print ("                             ")
print(classification_report(y_test, y_pred))

Accuracy Score
0.824
-----------------------------
Confusion Matrix
                             
          Predicted 0  Predicted 1
Actual 0           23           14
Actual 1            8           80
-----------------------------
Classification Report
                             
              precision    recall  f1-score   support

           0       0.74      0.62      0.68        37
           1       0.85      0.91      0.88        88

    accuracy                           0.82       125
   macro avg       0.80      0.77      0.78       125
weighted avg       0.82      0.82      0.82       125

